### ...in which we read all the observation data (baselines, whatev...) and calculate quantities for comparison with model outputs and write to new NC files

In [1]:
import xarray as xr
import pathlib as pl
import numpy as np
import pandas as pd
import pywatershed

### define the model 

In [2]:
all_models = ['01473000', '05431486','09112500','14015000']

In [3]:
rootdir = pl.Path('../NHM_extractions/20230110_pois_haj/')

### make observations dirs in each extraction directory

In [4]:
for cm in all_models:
    if not (rootdir / cm / 'observation_data').exists():
        (rootdir / cm / 'observation_data').mkdir()

### now grab all the `nhm_ids` from the `myparam.param` file for each cutout

In [5]:
nhm_ids = dict(zip(all_models, 
        [pywatershed.parameters.PrmsParameters.load(rootdir / cm / 'myparam.param').parameters['nhm_id'] for cm in all_models]))

In [6]:
nhm_ids

{'01473000': array([5621, 5625, 5628, 5635, 5637, 5643, 5678, 5679, 5686, 5690, 5693,
        5697, 5703, 5728, 7128, 7156, 7157]),
 '05431486': array([36895, 36899, 36901, 36902, 36903, 36906, 36907, 36917, 36931,
        36962, 37892, 37895, 37896, 37985, 37986, 37988]),
 '09112500': array([84012, 84017, 84023, 84032, 84038, 84124, 84148, 84165, 85114,
        85116]),
 '14015000': array([98861, 98876, 98877, 98884, 98901, 98903, 98904, 98905, 98907,
        98915, 98936, 98937, 98940, 98941, 99581, 99618])}

### assign `wkdir` to indicate where the raw CONUS netCDF files live

In [7]:
wkdir = pl.Path('../Supporting_information/CONUS_baselines')

In [8]:
lu = pd.read_csv('../Supporting_information/target_and_output_vars_table.csv', index_col=0)
lu

,output variable,time aggregation,calibration period
target (baseline) id,,,
aet_min,hru_actet,monthly,2000–2010
aet_max,hru_actet,monthly,2000–2010
aet_min,hru_actet,mean monthly*,2000–2010
aet_max,hru_actet,mean monthly*,2000–2010
runoff_min,hru_outflow,monthly,1982–2010
runoff_max,hru_outflow,monthly,1982–2010
runoff_mwbm,hru_outflow,monthly,1982–2010
recharge_min_norm,recharge,annual,2000–2009
recharge_max_norm,recharge,annual,2000–2009


In [9]:
[i for i in wkdir.glob('*.nc')]

[WindowsPath('../Supporting_information/CONUS_baselines/baseline_AET_v11.nc'),
 WindowsPath('../Supporting_information/CONUS_baselines/baseline_RCH_v11.nc'),
 WindowsPath('../Supporting_information/CONUS_baselines/baseline_RUN_v11.nc'),
 WindowsPath('../Supporting_information/CONUS_baselines/baseline_SCA_v11.nc'),
 WindowsPath('../Supporting_information/CONUS_baselines/baseline_SOMann_v11.nc'),
 WindowsPath('../Supporting_information/CONUS_baselines/baseline_SOMmth_v11.nc')]

### Handle the AET parameters first

In [10]:
AET_all = xr.open_dataset(wkdir / 'baseline_AET_v11.nc')
AET_all

<xarray.Dataset>
Dimensions:     (time: 132, nhru: 114958)
Coordinates:
  * time        (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2010-12-01
  * nhru        (nhru) int32 1 2 3 4 5 6 ... 114954 114955 114956 114957 114958
Data variables:
    nhm_id      (nhru) int32 ...
    aet_mwbm    (time, nhru) float32 ...
    aet_mod16   (time, nhru) float32 ...
    aet_ssebop  (time, nhru) float32 ...
    aet_max     (time, nhru) float32 ...
    aet_min     (time, nhru) float32 ...
Attributes:
    Description:  Baseline monthly evapotranspiration

In [11]:
for cm, c_ids in nhm_ids.items():
    c_da = AET_all.sel(nhru=c_ids)
    c_da[['aet_max','aet_min']].to_netcdf(rootdir / cm / 'observation_data' / f'AET_monthly.nc')
    c_da.groupby('time.month').mean().to_netcdf(rootdir / cm / 'observation_data' / f'AET_mean_monthly.nc')
AET_all.close()

###  runoff

In [12]:
RUN_all = xr.open_dataset(wkdir / 'baseline_RUN_v11.nc')
RUN_all

<xarray.Dataset>
Dimensions:      (time: 708, nhru: 114958)
Coordinates:
  * time         (time) datetime64[ns] 1952-01-01 1952-02-01 ... 2010-12-01
  * nhru         (nhru) int32 1 2 3 4 5 6 ... 114954 114955 114956 114957 114958
Data variables:
    nhm_id       (nhru) int32 ...
    runoff_mwbm  (time, nhru) float32 ...
    runoff_max   (time, nhru) float32 ...
    runoff_min   (time, nhru) float32 ...
Attributes:
    Description:  Baseline monthly runoff

In [13]:
for cm, c_ids in nhm_ids.items():
    c_da = RUN_all.sel(nhru=c_ids, time=slice('1982-01-01','2010-01-01'))
    c_da[['runoff_mwbm','runoff_min', 'runoff_max']].to_netcdf(rootdir / cm / 'observation_data' / f'RUN_monthly.nc')
RUN_all.close()

### recharge annual
### QUESTION - should these really be summed???

In [14]:
RCH_all = xr.open_dataset(wkdir / 'baseline_RCH_v11.nc')
RCH_all

<xarray.Dataset>
Dimensions:            (time: 10, nhru: 114958)
Coordinates:
  * time               (time) datetime64[ns] 2000-01-01 ... 2009-01-01
  * nhru               (nhru) int32 1 2 3 4 5 ... 114955 114956 114957 114958
Data variables:
    nhm_id             (nhru) int32 ...
    recharge_min_norm  (time, nhru) float32 ...
    recharge_max_norm  (time, nhru) float32 ...
Attributes:
    Description:  Baseline annual recharge

In [15]:
for cm, c_ids in nhm_ids.items():
    c_da = RCH_all.sel(nhru=c_ids)
    c_da[['recharge_min_norm','recharge_max_norm']].resample(time='1Y').sum().to_netcdf(rootdir / cm / 'observation_data' / f'RCH_annual.nc')
RCH_all.close()

### soil moisture --- annual

In [16]:
SOM_ann_all = xr.open_dataset(wkdir / 'baseline_SOMann_v11.nc')
SOM_ann_all

<xarray.Dataset>
Dimensions:              (time: 29, nhru: 114958)
Coordinates:
  * time                 (time) datetime64[ns] 1982-01-01 ... 2010-01-01
  * nhru                 (nhru) int32 1 2 3 4 5 ... 114955 114956 114957 114958
Data variables:
    nhm_id               (nhru) int32 ...
    soil_moist_min_norm  (time, nhru) float32 ...
    soil_moist_max_norm  (time, nhru) float32 ...
Attributes:
    Description:  Baseline annual soil moisture

In [17]:
for cm, c_ids in nhm_ids.items():
    c_da = SOM_ann_all.sel(nhru=c_ids)
    c_da[['soil_moist_min_norm','soil_moist_max_norm']].to_netcdf(rootdir / cm / 'observation_data' / f'Soil_Moisture_annual.nc')
SOM_ann_all.close()

### soil moisture --- monthly

In [18]:
SOM_mon_all = xr.open_dataset(wkdir / 'baseline_SOMmth_v11.nc')
SOM_mon_all

<xarray.Dataset>
Dimensions:              (time: 348, nhru: 114958)
Coordinates:
  * time                 (time) datetime64[ns] 1982-01-01 ... 2010-12-01
  * nhru                 (nhru) int32 1 2 3 4 5 ... 114955 114956 114957 114958
Data variables:
    nhm_id               (nhru) int32 ...
    soil_moist_min_norm  (time, nhru) float32 ...
    soil_moist_max_norm  (time, nhru) float32 ...
Attributes:
    Description:  Baseline monthly soil moisture

In [19]:
for cm, c_ids in nhm_ids.items():
    c_da = SOM_mon_all.sel(nhru=c_ids)
    c_da[['soil_moist_min_norm','soil_moist_max_norm']].to_netcdf(rootdir / cm / 'observation_data' / f'Soil_Moisture_monthly.nc')
SOM_mon_all.close()

### Snow cover --- daily
#### checked with Parker on 4/18/23 and script appears to function as intended. Also added more explaination.

In [20]:
# Read the raw data set. Lauren Hay developed fortran code embedded in the NHM that pre-processed the raw data,
# applying several filters.
SCA= xr.open_dataset(wkdir / 'baseline_SCA_v11.nc')
SCA

<xarray.Dataset>
Dimensions:            (time: 7188, nhru: 114958)
Coordinates:
  * time               (time) datetime64[ns] 2000-01-01 ... 2019-09-05
  * nhru               (nhru) int32 1 2 3 4 5 ... 114955 114956 114957 114958
Data variables:
    nhm_id             (nhru) int32 ...
    snow_cover_extent  (time, nhru) float32 ...
    sca_clear_index    (time, nhru) float32 ...
Attributes:
    Description:  Baseline daily snowcovered area

In [21]:
# populating variables used in Parker Norton's function.
baseline_file = wkdir / 'baseline_SCA_v11.nc'
sca_var = 'snow_cover_extent'
ci_var = 'sca_clear_index'
st_date = '2000-01-01' #per publication
en_date = '2010-12-31' #per publication
remove_ja = True #This is technically the first filter for removing July and August from the dataset

In [22]:
def get_dataset(filename, f_vars, start_date, end_date):
    # This routine assumes dimension nhru exists and variable nhm_id exists
    df = xr.open_dataset(filename)
    # NOTE: Next line needed if nhm_id variable exists in netcdf file
    df = df.assign_coords(nhru=df.nhm_id)
    if isinstance(f_vars, list):
     df = df[f_vars].sel(time=slice(start_date, end_date))
    else:
     df = df[[f_vars]].sel(time=slice(start_date, end_date))
    return df

baseline_df = get_dataset(baseline_file, [sca_var, ci_var, 'nhru'], st_date, en_date) 

#Applying first filter to remove selected months, July and August, from the dataset, selects months to keep.
if remove_ja:
    # 
    baseline_restr = baseline_df.sel(time=baseline_df.time.dt.month.isin([1, 2, 3, 4, 5, 6, 9, 10, 11, 12]))
else:
    baseline_restr = baseline_df
baseline_df.close()     

In [23]:
# Create the SCAmask to remove other data meeting criteria below.

# Compute lower and upper SCA values based on confidence interval(used to be called the clear index). Comes from MODIS,
# "fraction of the cell observed in cloud free conditions," here, if cloud cover is less than 30%, then,
# the SCA values is used; 
threshold = 70.0
ci_pct = baseline_restr[ci_var].where(baseline_restr[ci_var] >= threshold)
ci_pct /= 100.0

# Mask SCA values where CI is masked; this included daily targets for HRUs when the clear index was greater than 70%
sca_obs = baseline_restr[sca_var].where(~np.isnan(ci_pct))

# Maximum SCA value of those within the threshold...so really "sca_obs_max"
msk_SCAmax = sca_obs.max(axis=0)

# Now count the data sca_obs:
# Number of daily values > 0.0 by HRU
msk_num_obs = (sca_obs > 0.0).sum(axis=0)

#Excluding HRUs that do not have enough values
#Number of years of values by HRU: How many years of annula values that are greater than 0?
msk_num_ann = sca_obs.resample(time='1AS').mean() # resamples the df and finds the average value for each year
msk_num_ann = (msk_num_ann > 0).sum(axis=0) # takes a count of all average annual values greater than 0.

# Create SCA mask based on:
# 1 - Keeps HRUs targets where at least 2 years of data that were the annual average values are greater than 0 (see above),
# 2 - and, where sca_max is greater than 50%, 
# 3 - and, where there are least 9 days of values in the total selected period.
SCAmask = (msk_num_ann > 1) & (msk_SCAmax > 0.5) & (msk_num_obs > 9)

In [24]:
# Lower bound of SCA by HRU
baseline_SCAmin = (ci_pct * sca_obs).where(SCAmask)# Computes min based upon %SCA of the %area visible.

In [25]:
# Upper bound of SCA by HRU
baseline_SCAmax = (baseline_SCAmin + (1.0 - ci_pct)).where(SCAmask)# Computes max based upon % SCA + %area not visible

In [26]:
SCA_daily = xr.combine_by_coords([baseline_SCAmin.to_dataset(name='SCA_min'), baseline_SCAmax.to_dataset(name='SCA_max')])
SCA_daily

<xarray.Dataset>
Dimensions:  (nhru: 114958, time: 3336)
Coordinates:
  * nhru     (nhru) int32 1 2 3 4 5 6 ... 114954 114955 114956 114957 114958
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2010-12-31
Data variables:
    SCA_max  (time, nhru) float32 nan nan nan nan ... 0.9909 0.8565 0.2741
    SCA_min  (time, nhru) float32 nan nan nan nan ... 0.9827 0.9816 0.6828 0.219

In [27]:
for cm, c_ids in nhm_ids.items():
    c_da = SCA_daily.sel(nhru=c_ids)
    c_da.to_netcdf(rootdir / cm / 'observation_data' / f'SCA_daily.nc')

In [28]:
SCA.close()
SCA_daily.close()